In [2]:
from allennlp.modules.elmo import Elmo, batch_to_ids
from sacremoses import MosesTokenizer, MosesDetokenizer

options_file = "elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
weights_file = "elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
model = Elmo(options_file, weights_file, 1, dropout=0).eval()
tokenizer = MosesTokenizer()

In [3]:
sequences = ["Is this an ELMo representation? Yes", "Is this an ELMo representation? No"]
tokens = [tokenizer.tokenize(sequence, escape=False) for sequence in sequences]
character_ids = batch_to_ids(tokens)
character_ids

tensor([[[259,  74, 116, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259, 117, 105, 106, 116, 260, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259,  98, 111, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259,  70,  77,  78, 112, 260, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 26

In [4]:
embeddings = model(character_ids)
representations = embeddings["elmo_representations"][0]
representations

tensor([[[-0.8412, -0.1165,  0.5267,  ...,  0.4512, -0.1392,  0.2720],
         [-0.6446, -0.0498,  0.3769,  ...,  0.4740, -0.1965,  0.0583],
         [-0.2392,  1.1553,  0.1504,  ..., -0.0441, -0.4352,  0.2593],
         ...,
         [-0.3401, -0.1522,  0.0504,  ...,  0.1691, -0.0028, -0.2495],
         [-0.1372,  0.0519,  0.0663,  ...,  1.1651,  0.6015, -0.1530],
         [-0.2489, -0.0674,  0.5087,  ...,  0.5525, -0.2275,  0.5592]],

        [[-0.8412, -0.1165,  0.5267,  ...,  0.5106, -0.0239,  0.3420],
         [-0.6446, -0.0498,  0.3769,  ...,  0.5673, -0.0058,  0.2063],
         [-0.2392,  1.1553,  0.1504,  ..., -0.0058, -0.2897,  0.3701],
         ...,
         [-0.3401, -0.1522,  0.0504,  ...,  0.1757, -0.0016,  0.1202],
         [-0.1372,  0.0519,  0.0663,  ...,  1.1888,  0.6638, -0.0508],
         [ 0.0425,  0.0044,  0.2477,  ...,  0.1274, -0.0031,  0.7462]]],
       grad_fn=<CopySlices>)

In [5]:
yes_rep = representations[0]
print(yes_rep[0])
no_rep = representations[1]
print(no_rep[0])

tensor([-0.8412, -0.1165,  0.5267,  ...,  0.4512, -0.1392,  0.2720],
       grad_fn=<SelectBackward0>)
tensor([-0.8412, -0.1165,  0.5267,  ...,  0.5106, -0.0239,  0.3420],
       grad_fn=<SelectBackward0>)


### HuggingFace Interface

In [6]:
from allennlp.modules.elmo import Elmo, batch_to_ids
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers.modeling_outputs import SequenceClassifierOutput

class ElmoModel(PreTrainedModel):
    def __init__(self, options_file, weights_file):
        super().__init__(config=PretrainedConfig())
        self.elmo_model = Elmo(options_file, weights_file, 1, dropout=0)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, labels=None):
        return self.elmo_model(input_ids)["elmo_representations"][0]
    
    @staticmethod
    def from_pretrained(path):
        return ElmoModel(options_file, weights_file)

class ElmoTokenizer(PreTrainedTokenizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tokenizer = MosesTokenizer()
    
    def __call__(self, text):
        batch = text if isinstance(text, (list, tuple)) else [text]
        tokens = [self.tokenizer.tokenize(sequence, escape=False) for sequence in sequences]
        character_ids = batch_to_ids(tokens)
        return character_ids
    
    @staticmethod
    def from_pretrained(path):
        return ElmoTokenizer()


tokenizer = ElmoTokenizer.from_pretrained("elmo")
inputs = ["Is this an ELMo representation? Yes", "Is this an ELMo representation? No"]
character_ids = tokenizer(inputs)
model = ElmoModel.from_pretrained("elmo")
embeddings = model(character_ids)
    

In [8]:
embeddings[0].shape

torch.Size([7, 1024])